In [1]:
import boto3

session=boto3.Session(
    aws_access_key_id='',
    aws_secret_access_key='',
    region_name=''        
)

client=session.client('rds')

In [67]:
db_instance_identifier=''

## Creating DB Instance

In [68]:
def create_db_inst():
    
    response=client.create_db_instance(
        DBInstanceIdentifier = db_instance_identifier,
        DBInstanceClass = 'db.t2.micro',
        Engine = 'mysql',
        MasterUsername = 'admin',
        VpcSecurityGroupIds=['sg-0e8625dd6585b3f06'],
        #Virtual Private Cloud (VPC): VPC is a logically isolated section of a cloud provider's network
        #Subnet(Public(ec2), Private(RDS)): A subnet is a range of IP addresses in your VPC. Subnets are used to logically divide your VPC's IP address space into smaller segments
        MasterUserPassword = 'Rds-2023',
        AllocatedStorage = 20,  
        AvailabilityZone = 'ap-south-1a',  
        DBParameterGroupName = 'default.mysql8.0',  
        MultiAZ = False,
        Tags=[{'Key': 'Name', 'Value': 'MySqlDBInstance'}], 
    )

    return response

In [69]:
response=create_db_inst()
print(response)

client.get_waiter('db_instance_available').wait(DBInstanceIdentifier=db_instance_identifier)

# Print the DB instance details
print("DB instance created successfully!")
print("DB Instance ID:", response['DBInstance']['DBInstanceIdentifier'])
# print("Endpoint:", response['DBInstance']['Endpoint']['Address'])

{'DBInstance': {'DBInstanceIdentifier': 'mysql-db-instance-lab5', 'DBInstanceClass': 'db.t2.micro', 'Engine': 'mysql', 'DBInstanceStatus': 'creating', 'MasterUsername': 'admin', 'AllocatedStorage': 20, 'PreferredBackupWindow': '22:28-22:58', 'BackupRetentionPeriod': 1, 'DBSecurityGroups': [], 'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-0e8625dd6585b3f06', 'Status': 'active'}], 'DBParameterGroups': [{'DBParameterGroupName': 'default.mysql8.0', 'ParameterApplyStatus': 'in-sync'}], 'AvailabilityZone': 'ap-south-1a', 'DBSubnetGroup': {'DBSubnetGroupName': 'default', 'DBSubnetGroupDescription': 'default', 'VpcId': 'vpc-05ca77928f4caa30e', 'SubnetGroupStatus': 'Complete', 'Subnets': [{'SubnetIdentifier': 'subnet-0ca3117f6f6f2428b', 'SubnetAvailabilityZone': {'Name': 'ap-south-1c'}, 'SubnetOutpost': {}, 'SubnetStatus': 'Active'}, {'SubnetIdentifier': 'subnet-035e4c9c69bead117', 'SubnetAvailabilityZone': {'Name': 'ap-south-1a'}, 'SubnetOutpost': {}, 'SubnetStatus': 'Active'}, {'SubnetIden

In [48]:
print("Endpoint:", response['DBInstance']['Endpoint']['Address'])
#mysql-db-instance-lab5.co0ynpevyflj.ap-south-1.rds.amazonaws.com

KeyError: 'Endpoint'

## Create EC2 Instance

In [4]:
ec2 = session.resource('ec2')

instance = ec2.create_instances(
    ImageId='ami-0ded8326293d3201b',  
    InstanceType='t2.micro',
    MinCount=1,
    MaxCount=1,
    SecurityGroups=['launch-wizard-1'],  
    KeyName='CS351_KEY',
    UserData=open('startup_script.sh', 'r').read(),
)[0]

instance.create_tags(Tags=[{'Key': 'Name', 'Value': 'linux-db-lab5'}])

print(instance)
print('Running and waiting for DNS')
instance.wait_until_running()


public_dns = instance.public_dns_name
print("Public DNS:", public_dns)

ec2.Instance(id='i-08ca0069ac6ea17d2')
Running and waiting for DNS
Public DNS: ec2-65-2-137-221.ap-south-1.compute.amazonaws.com


## Create Flask File

In [3]:
!pip install mysql.connector pymysql

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 2.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for mysql.connector: filename=mysql_connector-2.2.9-cp311-cp311-linux_x86_64.whl size=247950 sha256=fd068ef9295523a6b02cdfff53ae4f3c7a4e4f4e4bc16771d8b92973bb7dbd19
  Stored in directory: /home/av/.cache/pip/wheels/17/cd/ed/2d49e9bac69cf09382e4c7cc20a2511202b48324b87db26019
Successfully built mysql.connector


In [ ]:

import pymysql

# Configure database connection
db_host = ''
db_user = ''
db_password = ''
db_name = ''


db = pymysql.connect(
        host=db_host,
        user=db_user,
        password=db_password,
        database=db_name,
        port=3306
    )

name = input("Enter your name: ")
message = input("Enter your message: ")

cursor = db.cursor()
insert_query = "INSERT INTO comments (name, message) VALUES (%s, %s)"
cursor.execute(insert_query, (name, message))
db.commit()
cursor.close()
db.close()
